In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from pathlib import Path  
import glob

In [5]:
# directory_path = "/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls"
directory_path = "/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016"

In [6]:
text_files = glob.glob(f"{directory_path}/*.xml")
text_files

['/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7331107_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7307951_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7330709_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7331098_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7328508_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7325241_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7332866_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7330148_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7325240_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7328416_T.xml',
 '/Users/diliyayalikun/Downloads/Diliya Work/src/data/earnings_calls/2016/7308156_T.xml',
 '/Users/d

In [7]:
text_titles = [Path(text).stem for text in text_files]
text_titles

['7331107_T',
 '7307951_T',
 '7330709_T',
 '7331098_T',
 '7328508_T',
 '7325241_T',
 '7332866_T',
 '7330148_T',
 '7325240_T',
 '7328416_T',
 '7308156_T',
 '7325242_T',
 '7307299_T']

In [8]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english',ngram_range=(2,2))
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names_out())

In [8]:
# tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()
# tfidf_slice = tfidf_df[['earning', 'transcript', 'increase', 'conference', 'lost', 'stocks','tax', 'men', 'women', 'children']]
# tfidf_slice.sort_index().round(decimals=2)

In [9]:
# tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

In [9]:

tfidf_df.stack().reset_index()
# tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
# tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,level_0,level_1,0
0,7331107_T,00 00am,0.000000
1,7331107_T,00 00pm,0.000000
2,7331107_T,00 10pm,0.000000
3,7331107_T,00 inaudible,0.022006
4,7331107_T,00 stock,0.022006
...,...,...,...
317533,7307299_T,ziggo upc,0.000000
317534,7307299_T,zone data,0.000000
317535,7307299_T,zone rar,0.000000
317536,7307299_T,zone really,0.000000


In [10]:
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

In [11]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,document,term,tfidf
295990,7307299_T,banking group,0.369507
306012,7307299_T,lloyds banking,0.369507
303241,7307299_T,group plc,0.361109
302674,7307299_T,george culmer,0.201549
299216,7307299_T,culmer lloyds,0.193151
...,...,...,...
161571,7332866_T,nv ceo,0.171250
149915,7332866_T,birgit conix,0.088259
165552,7332866_T,rob goyens,0.088259
152129,7332866_T,conix telenet,0.077226


In [12]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)
top_tfidf[top_tfidf['term'].str.contains('b2b')]

,document,term,tfidf


In [13]:
top_tfidf[top_tfidf['document'].str.contains('7330148')]

,document,term,tfidf
179989,7330148_T,fiscal 2016,0.176335
181079,7330148_T,gross margin,0.152065
181416,7330148_T,hardware holdings,0.132251
189678,7330148_T,restoration hardware,0.132251
189864,7330148_T,rh modern,0.132251
192208,7330148_T,supply chain,0.132251
183959,7330148_T,long term,0.105024
177157,7330148_T,customer experience,0.095041
180234,7330148_T,fourth quarter,0.095041
190923,7330148_T,short term,0.095041


In [14]:
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)